In [16]:
import sys; sys.path.append('..')
from osp import *

In [17]:
get_current_feat_weights()

,weight,run,feat_desc,score_mean1,score_mean2,score_z1,score_z2,score_mean_diff,score_mean_diff_abs,score_mean_diff_pct,score_mean_div,score_mean_div_abs,score_z_diff,score_z_diff_abs,score_z_diff_pct,score_z_div,score_z_div_abs
feature,,,,,,,,,,,,,,,,,
deprel_acl,0.021298,4.5,NaN,10.866098,9.019650,10.825173,9.019650,1.846448,1.846448,0.202927,1.202927,1.202927,1.805522,1.805522,0.198267,1.198267,1.198267
deprel_acl:relcl,-0.013912,4.5,NaN,14.124723,12.251148,14.174844,12.251148,1.873576,1.873576,0.165178,1.165178,1.165178,1.923696,1.923696,0.168952,1.168952,1.168952
deprel_advcl,-0.005100,4.5,NaN,18.840961,16.309650,18.775265,16.309650,2.531310,2.531310,0.154477,1.154477,1.154477,2.465615,2.465615,0.150474,1.150474,1.150474
deprel_advcl:relcl,0.096775,4.5,NaN,0.572644,0.415064,0.566557,0.415064,0.157581,0.157581,0.355600,1.355600,1.355600,0.151494,0.151494,0.342293,1.342293,1.342293
deprel_advmod,0.022723,4.5,NaN,54.592577,48.595370,54.594193,48.595370,5.997208,5.997208,0.123798,1.123798,1.123798,5.998823,5.998823,0.123843,1.123843,1.123843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sent_height,0.005876,4.5,NaN,496.628185,462.580322,497.209402,462.580322,34.047863,34.846650,0.074386,1.074386,1.074386,34.629080,35.567024,0.075640,1.075640,1.075640
sent_num_dependent_clauses,0.029270,4.5,NaN,45.429632,34.044279,45.491494,34.044279,11.385353,11.385353,0.333396,1.333396,1.333396,11.447214,11.447214,0.335152,1.335152,1.335152
sent_num_independent_clauses,-0.005695,4.5,NaN,58.119852,58.514766,58.098737,58.514766,-0.394914,2.230784,-0.005602,0.994398,0.994398,-0.416029,2.236637,-0.005968,0.994032,0.994032


In [18]:
for id,docstr in STASH_SLICES_NLP.items():
    break
doc=stanza.Document.from_serialized(docstr)
sent = random.choice(doc.sentences)
sent.text

'But I deny that I can abstract one from another, or conceive separately, those qualities which it is impossible should exist so separated; or that I can frame a general notion by abstracting from particulars in the manner aforesaid.'

In [19]:
def is_leaf(tree):
    """Check if a tree node is a leaf (preterminal) node."""
    return not isinstance(tree[0], nltk.Tree) if len(tree) > 0 else False


def get_leaf_ids(tree):
    print(tree)
    return [word.id for word in tree.leaves()]

def iter_preterms(tree):
    for t in tree.subtrees():
        if is_leaf(t):
            yield t

def tokenize_clauses(tree): # -> return both whether "IC" or "DC"
    clause_num = 0
    clause_type = None
    for t in tree.subtrees():
        ttxt = ' '.join(t.leaves())
        if ttxt and ttxt[0].isalpha() and ((not is_in_sbar(t) and t.label() == 'S') or t.label() == 'SBAR'):
            clause_num += 1
            clause_type = 'DC' if is_in_sbar(t) else 'IC'

        t._clause_num = clause_num
        t._clause_type = clause_type

    # o = defaultdict(list)
    o = []
    for preterm_i,preterm in enumerate(iter_preterms(tree)):
        d = {
            'clause_num': preterm._clause_num,
            'clause_type': preterm._clause_type,
            'word_num': preterm_i+1,
            'word': ' '.join(preterm.leaves())
        }
        o.append(d)
    
    odf = pd.DataFrame(o)
    odf['clause_num'] = odf['clause_num'].rank(method='dense', ascending=True).apply(int)
    return odf
    

In [20]:
# sent = get_nlp_doc("The trouble about progress is that it always looks much greater than it really is.").sentences[0]
# # sent

In [21]:
def get_clauses(sent):
    if isinstance(sent, str):
        sent = get_nlp_doc(sent).sentences[0]
    tree = get_sent_tree(sent)
    odf = tokenize_clauses(tree)
    assert len(odf) == len(sent.words)
    odf['pos'] = [w.xpos for w in sent.words]
    odf['deprel'] = [w.deprel for w in sent.words]
    odf['head'] = [w.head for w in sent.words]
    return odf.set_index(['clause_num','clause_type','word_num'])

In [22]:
df = get_clauses(sent)

In [24]:
# df_feats.loc['deprel_mark']

In [117]:
# def get_color(
#     feat,
#     color_by='weight',
#     df_feats=None,
# ):
#     if df_feats is None:
#         df_feats = get_current_feat_weights()
    
#     featval = df_feats.loc[feat, color_by] if feat in df_feats.index else 0
#     color_by_min = float(df_feats[color_by].fillna(0).min())
#     color_by_max = float(df_feats[color_by].fillna(0).max())
    
#     # Normalize score to [-1, 1] range
#     if color_by_max == color_by_min:
#         intensity = 0
#     else:
#         intensity = max(color_by_min, min(color_by_max, featval))
    
#     # Map score to color: Blue (positive) to Orange (negative)
#     if intensity > 0:
#         # Positive score -> Blue
#         return f'rgba(0, 0, 255, {intensity:.2f})'
#     elif intensity < 0:
#         # Negative score -> Orange
#         return f'rgba(255, 165, 0, {-intensity:.2f})'
#     else:
#         return 'rgba(0, 0, 0, 0)'

In [214]:
# from colour import Color
# color_range = list(Color('red').range_to(Color('gray'), 50)) + list(Color('gray').range_to(Color('red'), 50))
# # list(color_range)

# def get_color(
#     feat,
#     color_by='weight',
#     df_feats=None,
# ):
#     if df_feats is None:
#         df_feats = get_current_feat_weights()
#     featval = df_feats.loc[feat, color_by] if feat in df_feats.index else 0
#     # color_by_min = float(df_feats[color_by].fillna(0).min())
#     # color_by_max = float(df_feats[color_by].fillna(0).max())
#     color_by_min = -.5
#     color_by_max = .5
#     color_by_range = color_by_max - color_by_min
#     color_idx = int(100 * (featval - color_by_min) / color_by_range)
#     if color_idx < 0:
#         color_idx = 0
#     elif color_idx > 99:
#         color_idx = 99
#     print(feat,color_idx,color_by_min,color_by_max)
#     try:
#         color = color_range[color_idx]
#         color.saturation = 1.0
#         # color.luminance = 0.5
#         # color.hue = 0.0
#         return color.hex
#     except:
#         return '#000000'

In [215]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

def get_color(
    feat,
    color_by='weight',
    df_feats=None,
    cmap_name='RdBu'  # 'RdBu' is a standard Red-to-Blue diverging colormap
):
    if df_feats is None:
        df_feats = get_current_feat_weights()
    
    # 1. Get the value for the feature
    val = df_feats.loc[feat, color_by] if feat in df_feats.index else 0
    
    # 2. Determine the range dynamically from the data
    v_min = df_feats[color_by].min()
    v_max = df_feats[color_by].max()
    
    # Handle edge case where all values are the same
    if v_min == v_max:
        return '#808080' # Return gray if no range
    
    # 3. Create a normalization object and map to the colormap
    # Note: RdBu usually has Red for high and Blue for low, 
    # use 'RdBu_r' if you want Red for low and Blue for high.
    norm = mcolors.Normalize(vmin=v_min, vmax=v_max)
    cmap = plt.get_cmap(cmap_name)
    
    rgba = cmap(norm(val))
    return mcolors.to_hex(rgba)

In [216]:
get_color('pos_NNPS',color_by='weight')

'#dc6e57'

In [217]:
def get_manual_displacy_data(clause_df):
    clause_data = {
        "words": [{"text": text, "tag": pos} for text,pos in zip(clause_df['word'], clause_df['pos'])],
        "arcs": [
            {
                "start": row['word_num']-1,
                "end": row['head']-1,
                "label": row['deprel'],
                "dir": "left" if row['word_num'] > row['head'] else "right"
            }
            for _, row in clause_df.reset_index().iterrows()
            if row['word_num'] < row['head']
        ]
    }
    return clause_data

In [218]:
clause_data = get_manual_displacy_data(df)

In [219]:
# for clause, clause_df in df.groupby('clause_num'):
#     clause_data = get_manual_displacy_data(clause_df)
#     displacy.render(clause_data, style="dep", manual=True)
#     # x.render_to_file(f"clause_{clause}.svg")

In [220]:
# !pip install beautifulsoup4
from bs4 import BeautifulSoup

In [221]:
color_by = 'weight'

from IPython.display import HTML, display
from spacy import displacy
html_output = displacy.render(clause_data, style="dep", manual=True,options={'compact': True},jupyter=False)

# Parse the SVG
soup = BeautifulSoup(html_output, 'html.parser')

tag_elements = soup.find_all('tspan', class_='displacy-tag')
for idx, tag_elem in enumerate(tag_elements):
    tag_elem['fill'] = get_color(f'pos_{tag_elem.text}', color_by=color_by)

deprel_elements = soup.find_all('textpath')
for idx, deprel_elem in enumerate(deprel_elements):
    deprel_elem['fill'] = get_color(f'deprel_{deprel_elem.text}', color_by=color_by)

# # Combine and display
HTML(str(soup))
# # deprel_elements

In [213]:
# print(html_output )

In [17]:
stopxx

NameError: name 'stopxx' is not defined

In [16]:
displacy.serve(clause_data, style="dep", manual=True, port=5001)


Using the 'dep' visualizer
Serving on http://0.0.0.0:5001 ...



127.0.0.1 - - [25/Dec/2025 00:29:25] "GET / HTTP/1.1" 200 11108
127.0.0.1 - - [25/Dec/2025 00:29:25] "GET /favicon.ico HTTP/1.1" 200 11108


Shutting down server on port 5001.


In [17]:
# Format your Stanza data into this specific dictionary structure
manual_data = {
    "words": [
        {"text": "Barack", "tag": "PROPN"},
        {"text": "Obama", "tag": "PROPN"},
        {"text": "was", "tag": "AUX"},
        {"text": "born", "tag": "VERB"}
    ],
    "arcs": [
        {"start": 0, "end": 1, "label": "flat", "dir": "left"},
        {"start": 1, "end": 3, "label": "nsubjpass", "dir": "left"},
        {"start": 2, "end": 3, "label": "auxpass", "dir": "left"}
    ]
}

displacy.render(manual_data, style="dep", manual=True)

In [ ]:
claus

In [26]:
def get_displacy_data_from_sent(sent):
    words = [{"text": word.text, "tag": word.xpos} for word in sent.words]
    arcs = []
    
    for word in sent.words:
        # Skip root dependencies (head == 0)
        if word.head == 0:
            continue
        
        # Convert from 1-based (Stanza) to 0-based (displaCy) indexing
        start = word.head - 1  # head position (0-based)
        end = word.id - 1      # current word position (0-based)
        
        # Skip self-loops (word pointing to itself)
        if start == end:
            continue
        
        # Determine direction
        if start < end:
            direction = "left"
        else:
            direction = "right"
            # Swap start and end for right-pointing arcs
            start, end = end, start
        
        arcs.append({
            "start": start,
            "end": end,
            "label": word.deprel,
            "dir": direction
        })
    
    return {"words": words, "arcs": arcs}

data = get_displacy_data_from_sent(sent)

In [27]:
displacy.render(data, style="dep", manual=True)